# Notebook to Implement Model Training 

---

### 1) Setup

In [44]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import balanced_accuracy_score, make_scorer
from sklearn.model_selection import RepeatedKFold, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [21]:
SEED = 42

In [2]:
TRAIN_CLINICAL_FILENAME = "train_set_clinical.csv"
TEST_CLINICAL_FILENAME = "test_set_clinical.csv"

---

### 2) Read and Preprocess Data

In [14]:
train = pd.read_csv(TRAIN_CLINICAL_FILENAME, sep=";", index_col="ID")

In [15]:
train.shape

(132, 650)

In [16]:
train.head()

,Age (Y),Sex,Fever,Cough,Headache,Sore throat,Muscle or Body Aches,Fadigue,Congestion or runny nose,Shortness of breath or difficulty breathing,...,Freq.8943.76551923189,Freq.9058.85825530971,Freq.9098.58510797401,Freq.9437.74469644083,Freq.9593.90405666006,Freq.9799.842201746,Freq.10432.4853106264,Freq.11006.9514551194,Freq.11161.31855876,Group
ID,,,,,,,,,,,,,,,,,,,,,
1,53,F,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.000017,0.000005,0.000020,0.000029,0.000037,0.000024,0.000016,0.000030,0.000030,MILD
2,21,F,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.000021,0.000013,0.000002,0.000004,0.000005,0.000002,0.000008,0.000103,0.000002,MILD
5,62,F,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.000099,0.000079,0.000013,0.000012,0.000001,0.000062,0.000003,0.000089,0.000028,MILD
6,34,F,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.000025,0.000021,0.000017,0.000010,0.000022,0.000156,0.000015,0.000053,0.000007,SEVERE
8,42,M,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.000009,0.000153,0.000006,0.000046,0.000003,0.000017,0.000008,0.000110,0.000007,SEVERE


In [17]:
##### Preprocessing

train_mod = train.copy()
# Drop NaN Values 
train_mod.dropna(inplace=True)
# Convert Sex column to boolean (Female: 1, Male: 0)
train_mod["Sex"] = np.where(train_mod["Sex"]=="F", 1, 0)

In [19]:
train_mod.shape

(131, 650)

In [18]:
train_mod.head()

,Age (Y),Sex,Fever,Cough,Headache,Sore throat,Muscle or Body Aches,Fadigue,Congestion or runny nose,Shortness of breath or difficulty breathing,...,Freq.8943.76551923189,Freq.9058.85825530971,Freq.9098.58510797401,Freq.9437.74469644083,Freq.9593.90405666006,Freq.9799.842201746,Freq.10432.4853106264,Freq.11006.9514551194,Freq.11161.31855876,Group
ID,,,,,,,,,,,,,,,,,,,,,
1,53,1,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.000017,0.000005,0.000020,0.000029,0.000037,0.000024,0.000016,0.000030,0.000030,MILD
2,21,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.000021,0.000013,0.000002,0.000004,0.000005,0.000002,0.000008,0.000103,0.000002,MILD
5,62,1,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.000099,0.000079,0.000013,0.000012,0.000001,0.000062,0.000003,0.000089,0.000028,MILD
6,34,1,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.000025,0.000021,0.000017,0.000010,0.000022,0.000156,0.000015,0.000053,0.000007,SEVERE
8,42,0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.000009,0.000153,0.000006,0.000046,0.000003,0.000017,0.000008,0.000110,0.000007,SEVERE


---

### 3) Baseline Model Training and CV

In [37]:
# Define Classifier (or pipeline)
clf = RandomForestClassifier(random_state=SEED)

In [38]:
# Get Features and Target
X, y = train_mod.drop("Group", axis=1), train_mod["Group"]

In [39]:
# Defining RepeatedKFold Cross Validator
rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=SEED)

In [40]:
# Define metric scorer
metric_scorer = make_scorer(balanced_accuracy_score)

In [41]:
# Cross validate model
scores = cross_val_score(clf, X, y, scoring=metric_scorer, cv=rkf, n_jobs=-1)

In [42]:
# Score from each CV Iteration
scores

array([0.6       , 0.7518797 , 0.76875   , 0.66339869, 0.77777778,
       0.61813187, 0.72222222, 0.74836601, 0.62727273, 0.69444444,
       0.61111111, 0.76923077, 0.54761905, 0.73308271, 0.6372549 ,
       0.75      , 0.65384615, 0.84722222, 0.67261905, 0.78571429,
       0.7       , 0.69230769, 0.7593985 , 0.66666667, 0.61111111,
       0.75      , 0.63636364, 0.55625   , 0.77272727, 0.65972222,
       0.74117647, 0.52857143, 0.81875   , 0.64285714, 0.72727273,
       0.75      , 0.55357143, 0.75      , 0.61111111, 0.65413534,
       0.72222222, 0.89166667, 0.67857143, 0.71875   , 0.64848485,
       0.58223684, 0.70606061, 0.70833333, 0.75238095, 0.64285714])

In [43]:
# Mean Metric Value
np.mean(scores)

0.692270009574189

---

### 4) Experiments

##### 4.1) Hyper Parameter Optimization with GridSearchCV

In [46]:
# Define hyper parameters search space
parameters = {
    "n_estimators": [100, 500, 1000],
    "max_depth": [5, 10, 20]
}

In [50]:
# Define new model (or pipeline) and Grid Search (could be Random Search or Optune as well) Object for HP Optimization
clf = RandomForestClassifier(random_state=SEED)
search = GridSearchCV(clf, parameters, scoring=metric_scorer, cv=rkf, verbose=3)

In [ ]:
search.fit(X, y)

In [53]:
# Best Score
search.best_score_

0.6998807091171644

In [52]:
# Best Params
search.best_params_

{'max_depth': 5, 'n_estimators': 100}